# ドロップアウト (Dropout)

In [ ]:
# PyTorchが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1

from torch import set_num_threads, set_num_interop_threads
num_threads = 1
set_num_threads(num_threads)
set_num_interop_threads(num_threads)

In [ ]:
#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchinfo
from tqdm import tqdm
import utils

## 過学習(Overtraining)の例
### データ点の生成
sin関数に0.3の大きさのガウシアンノイズがのったデータを考えます。データ数は30にします。
早期終了の例と同様に、scikit learningの関数を使ってデータをトレーニング用と検証用に分割します。

In [ ]:
# データ点の取得
x, t = utils.dataset_overtraining(n=30, noise_scale=0.3)

# 学習データと検証用データに分割
from sklearn.model_selection import train_test_split
x_train, x_valid, t_train, t_valid = train_test_split(x, t, test_size=0.30, random_state=0)

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# データ点をプロット
ax.scatter(x_train, t_train, s=10, c="black", label="training data")
ax.scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
x_grid = np.linspace(-np.pi, np.pi, 100)
ax.plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax.legend()

# 図を表示
plt.show()

# numpy -> torch.tensor
x_grid_tensor = torch.from_numpy(x_grid).float().unsqueeze(-1)
x_tensor = torch.from_numpy(x).float().unsqueeze(-1)
t_tensor = torch.from_numpy(t).float().unsqueeze(-1)
x_train_tensor = torch.from_numpy(x_train).float().unsqueeze(-1)
t_train_tensor = torch.from_numpy(t_train).float().unsqueeze(-1)
x_valid_tensor = torch.from_numpy(x_valid).float().unsqueeze(-1)
t_valid_tensor = torch.from_numpy(t_valid).float().unsqueeze(-1)

### デモに用いる深層学習モデル
過学習の様子を見るために、パラメータ数の多いモデルを使ってフィットをしてみます。
ここでは、ノード数が128の隠れ層を5層重ねた多層パーセプトロンを使用します。
活性化関数としてはReLUを使い、モデルの出力の直前のノードは、活性化関数を使用しないことにします。
誤差関数は二乗和誤差を使い、最適化関数としてadamを使用します。

In [ ]:
# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=1, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=1),
)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 300
for i_epoch in tqdm(range(num_epochs)):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid_tensor).detach().numpy(), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train_history, label="loss (train)")
ax[1].plot(loss_valid_history, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

## ドロップアウト (Dropout)

ドロップアウトは、学習時に、ランダムにノードを消すという操作をすることで、表現能力を抑制する手法です。
この操作は、アンサンブル学習を擬似的に行っていると見ることもできます。

この操作はPyTorchの関数を用いることで簡単に実装することができます。

https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html

```python
Dropout(p=0.3)
```
ここで、消すノードの割合を`p`で指定します。0だと、ノードは全く消されません。1.0だと、全てのノードが消されます。

ここでは、MLPの各層の後にドロップアウト層を入れることにします。

In [ ]:
# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=1, out_features=128),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(in_features=128, out_features=1),
)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 300
for i_epoch in tqdm(range(num_epochs)):
    # trainモードにすることで、dropoutがオンになります。
    model.train()

    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # evalモードにすることで、dropoutがオフになります。
    model.eval()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

# evalモードにすることで、dropoutをオフにします。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid_tensor).detach().numpy(), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train_history, label="loss (train)")
ax[1].plot(loss_valid_history, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

トレーニングデータに対する誤差関数が大きくふらついていることがわかります。
これは、消されるノードが毎回ランダムに選ばれるためです。
一方で、検証用データではばらつきは比較的小さくなっています。
これは、`model.eval()`を実行したことで、検証用データを使ってモデルの予測をする際は全てのノードを使うようになっているためです。

ドロップアウトの`p`の値を変えたり、ドロップアウトの位置を変えると、モデルの予測はどのように変化するでしょうか？